# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [5]:
import pandas as pd
import boto3
import sagemaker
import os

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [7]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagarism_project'

# upload all data to S3
data_location = sagemaker_session.upload_data(os.path.join(data_dir), key_prefix = prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

counties/kmeans-2020-04-03-10-16-14-344/output/model.tar.gz
counties/pca-2020-04-03-08-32-00-620/output/model.tar.gz
plagarism_project/.ipynb_checkpoints/multi_train-checkpoint.csv
plagarism_project/.ipynb_checkpoints/train-checkpoint.csv
plagarism_project/multi_test.csv
plagarism_project/multi_train.csv
plagarism_project/test.csv
plagarism_project/train.csv
plagarism_projecy/test.csv
plagarism_projecy/train.csv
sagemaker-pytorch-2020-03-31-21-00-23-637/source/sourcedir.tar.gz
sagemaker-pytorch-2020-03-31-21-10-27-831/source/sourcedir.tar.gz
sagemaker-pytorch-2020-03-31-21-10-40-596/debug-output/training_job_end.ts
sagemaker-pytorch-2020-03-31-21-10-40-596/output/model.tar.gz
sagemaker-pytorch-2020-03-31-21-10-40-596/source/sourcedir.tar.gz
sagemaker-pytorch-2020-03-31-22-01-55-973/source/sourcedir.tar.gz
sagemaker-pytorch-2020-03-31-22-02-16-714/debug-output/training_job_end.ts
sagemaker-pytorch-2020-03-31-22-02-16-714/output/model.tar.gz
sagemaker-pytorch-2020-03-31-22-02-16-714/sour

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [5]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import sys
import sagemaker_containers
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
  

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [22]:

# your import and estimator code, here
from sagemaker.pytorch import PyTorch

pytorch_estimator = PyTorch(entry_point='train.py', 
                            source_dir = 'source_pytorch',
                            role = role,
                            train_instance_count=1,
                            train_instance_type='ml.p2.xlarge',
                            hyperparameters = {
                                'hidden_dim':200,
                                'epochs' : 50
                            },
                            framework_version = '0.4.0'
                           )

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [39]:
%%time

# Train your estimator on S3 training data

pytorch_estimator.fit({'train':data_location})

2020-04-05 21:21:07 Starting - Starting the training job...
2020-04-05 21:21:08 Starting - Launching requested ML instances...
2020-04-05 21:22:06 Starting - Preparing the instances for training......
2020-04-05 21:22:57 Downloading - Downloading input data...
2020-04-05 21:23:25 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-05 21:23:45,418 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-05 21:23:45,420 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-05 21:23:45,434 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-04-05 21:23:48,495 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-04-05 21:23:48,752 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-04-05 21:23:48,753

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [ ]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
predictor = pytorch_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')


----------!CPU times: user 202 ms, sys: 17.1 ms, total: 219 ms
Wall time: 5min 31s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [57]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x.values).round().astype('int')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [58]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

1.0

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

<b> Answer</b> :  I haven't got any false positives and false negatives because the model was trained using less amount of data at first and was tested upon very less (only 25) amount of test data as well. 
Although I can certainly say that the model is not overfitting as it was, it would definately have poor perfromance on the test dataset than the train dataset. 

### Question 2: How did you decide on the type of model to use? 

<b> Answer</b>: As the model predicts only 2 choice either plagarized or not plagarized, 
I decided to use a feed-foreward Tabular Linear Model for Binary Classification with 1 input layer, 1 hidden layer and a Sigmoid activation layer to produce the probailities of the class labels being 0 or 1. 


----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [59]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [52]:
# deleting bucket, uncomment lines below

# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.object.all().delete()

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

## Extending the Functionalities:
### Predicting the Plagarism Categories
We will be using two **linear layers** in between which we will use a **ReLU** non-linear activation function. At last for the multiclass classification we are using a **Softmax** activation function. As the optimizer we have chosen the **Stochastic Gradient Descent** in order to perform sigmoid action or explaining the features using probabilites across the label range. 

In [141]:
!pygmentize source_pytorch/model2.py

# torch imports
import torch.nn.functional as F
import torch.nn as nn


## TODO: Complete this classifier
class MulticlassClassifier(nn.Module):
   

    ## TODO: Define the init function, the input params are required (for loading code in train.py to work)
    def __init__(self, input_features, hidden_dim, output_dim):
        """
        Initialize the model by setting up linear layers.
        Use the input parameters to help define the layers of your model.
        :param input_features: the number of input features in your training/test data
        :param hidden_dim: helps define the number of nodes in the hidden layer(s)
        :param output_dim: the number of outputs you want to produce
        """
        super(MulticlassClassifier, self).__init__()

        # define any initial layers, here
        self.linear1 = nn.Linear(in_features = input_features, out_features = hidden_dim)
        self.relu =  nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(in_features = hidden_

<h3> Uploading the Category data to S3 </h3>

In [15]:
data_dir2 = 'plagiarism_data_multi'   #local directory location
prefix2 = 'plagarism_project_2'       #bucket location
data_multi_train_location = sagemaker_session.upload_data(os.path.join(data_dir2), key_prefix = prefix2) #uploading the data to S3
data_multi_train_location            

's3://sagemaker-us-west-2-782510500637/plagarism_project_2'

<h3> Creating a new PyTorch Estimator </h3>

In [152]:
from sagemaker.pytorch import PyTorch

pytorch_estimator2 = PyTorch(entry_point='train2.py',          #referring to the newly added training script
                            source_dir = 'source_pytorch',        
                            role = role,
                            train_instance_count=1, 
                            train_instance_type='ml.m4.xlarge',
                            hyperparameters = {
                                'hidden_dim':400,             
                                'epochs' : 100, 
                                'lr':0.02                     #learining rate of the Stochastic Gradient Descent Optimizer function
                            },
                            framework_version = '0.4.0'
                           )

<h3> Training the model with the new data </h3>

In [119]:
%%time

pytorch_estimator2.fit({'train':data_multi_train_location})      #training the model with the new dataset containing the categories of plagarism as target variables

2020-04-06 15:15:31 Starting - Starting the training job...
2020-04-06 15:15:33 Starting - Launching requested ML instances......
2020-04-06 15:16:35 Starting - Preparing the instances for training......
2020-04-06 15:17:42 Downloading - Downloading input data...
2020-04-06 15:18:19 Training - Downloading the training image...
2020-04-06 15:18:49 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-06 15:18:50,528 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-06 15:18:50,553 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-04-06 15:18:50,557 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-04-06 15:18:50,799 sagemaker-containers INFO     Module train2 does not provide a setup.py. 
Generating setup.py
2020-04-06 15:18:50,800 sagemak

<h3> Deploying the Model </h3>

In [145]:
%%time
predictor2 = pytorch_estimator2.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Using already existing model: sagemaker-pytorch-2020-04-06-15-15-31-288


---------------!CPU times: user 294 ms, sys: 17.3 ms, total: 311 ms
Wall time: 7min 32s


<h3> Testing the Model </h3>

In [146]:
import os
test_data = pd.read_csv(os.path.join(data_dir2, 'multi_test.csv'), header=None, names=None)
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

* `Category` labels to numerical labels according to the following rules (a higher value indicates a higher degree of plagiarism):
    * 0 = `non`
    * 1 = `heavy`
    * 2 = `light`
    * 3 = `cut`

In [147]:
from sklearn.metrics import accuracy_score
import numpy as np
def get_multiclass_accuracy(endpoint, test_data, labels):
    test_y_preds = endpoint.predict(test_data.values)
    test_y_preds = np.argmax(np.abs(test_y_classes), axis=1)
    labels = np.asarray(labels)
    accuracy = accuracy_score(labels, test_y_preds)
    print("Predicted Categories are: \n")
    print(test_y_preds)
    print("Actual Categories are: \n")
    print(labels)
    print("Accuracy Obtained in the Test Dataset for Category Prediction: " , accuracy)
    

In [148]:
get_multiclass_accuracy(predictor2, test_x, test_y)

Predicted Categories are: 

[3 3 2 0 2 3 0 0 0 0 0 0 0 3 3 2 2 3 0 3 0 3 3 0 0]
Actual Categories are: 

[3 3 1 1 2 2 0 0 0 0 0 0 1 2 3 1 2 3 0 3 0 1 2 0 0]
Accuracy Obtained in the Test Dataset for Category Prediction:  0.68


In [142]:
predictor2.delete_endpoint()

<h3> Comment on Inference of the Plagarism Categories </h3>
<p> The multiclass classification requires even more training data for even better accuracy on the test dataset. As the model was only trained with only 70 examples with fewer examples of each category, it haven't been able to capture all the categories correctly in the test dataset. </p>

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!